In [ ]:
####Lets read a sample audio using librosa

In [ ]:
import librosa
audio_file_path = '../input/urbansound8k/fold1/101415-3-0-3.wav'

In [ ]:
librosa_audio_data, librosa_sample_rate = librosa.load(audio_file_path)

In [ ]:
#plotting the data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#original audio with 1 channel

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(librosa_audio_data)

In [ ]:
'''
Observation librosa converts the signal to Mono ie 1 channel
'''

In [ ]:
#reading with scipy

In [ ]:
from scipy.io import wavfile as wav

In [ ]:
wave_sample_rate, wave_audio = wav.read(audio_file_path)

In [ ]:
#check the audio

In [ ]:
wave_audio

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#audio with 2 channels

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(wave_audio)

In [ ]:
#Extraction of Features- frequency and time characteristics of sound

In [ ]:
mfcc_feat = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc = 40)

print(mfcc_feat.shape)

In [ ]:
mfcc_feat

In [ ]:
#Extract the MFCC for every audio file


In [ ]:
import os
import pandas as pd
import librosa 
from tqdm import tqdm
 

In [ ]:
audio_dataset_path = '../input/urbansound8k/'
metadata_path = pd.read_csv('../input/urbansound8k/UrbanSound8K.csv')

In [ ]:
metadata_path.head(10)

In [ ]:
#function for feature extraction

In [ ]:
def audio_feature_extraction(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfcc_feat = librosa.feature.mfcc(y=audio, sr= sample_rate, n_mfcc=40)
    mfcc_scaled_features = np.mean(mfcc_feat.T, axis =0)
    return mfcc_scaled_features

In [ ]:
# iterate through each file and extract features

In [ ]:
import numpy as np

In [ ]:
extracted_features = []
for index_num, row in tqdm(metadata_path.iterrows()):
    #print(row[1][-1])
    #break
    #../input/urbansound8k/fold1/101415-3-0-2.wav
    #file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row[1][5])+'/',str(row[1][0]))
    #print(file_name)
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row['slice_file_name']))
    final_class_labels = row['class']#row[1][-1]
    data = audio_feature_extraction(file_name)
    extracted_features.append([data, final_class_labels])

In [ ]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

In [ ]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
X.shape


In [ ]:
y


In [ ]:
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=43)

In [ ]:
X_train

In [ ]:
y

In [ ]:
#Model Building

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense,Activation, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
## No of classes
num_labels=y.shape[1]


In [ ]:
print(num_labels)

In [ ]:
#Building the layers
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
#Compile the models
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/urbansound_8k_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

In [ ]:
X_test[1].shape

In [ ]:
model.predict_classes(X_test)

In [ ]:
filename="../input/urbansound8k/fold1/101415-3-0-8.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=model.predict_classes(mfccs_scaled_features)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class

In [ ]:
# Thanks for reading .